In [42]:
import pandas as pd
import glob,os
import re


In [43]:
OMOP_FILES_PATH = 'codelists/OMOP/'
OUTPUT_PATH = 'codelists/output/omop/uk'

omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')
omop_relationships  = pd.read_csv(OMOP_FILES_PATH+'RELATIONSHIP.csv', sep='\t')
omop_concept_relationships  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT_RELATIONSHIP.csv', sep='\t')


omop_concepts

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1625449282.py:4: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  omop_concepts  = pd.read_csv(OMOP_FILES_PATH+'CONCEPT.csv', sep='\t')


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
1473553,33091,IQVIA CO-CONNECT,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473554,33092,IQVIA CO-CONNECT MIABIS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473555,33093,IQVIA CO-CONNECT TWINS,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN
1473556,33095,NHS Ethnic Category,Metadata,Vocabulary,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN


In [44]:
s='colorectal'
files = sorted( list( glob.glob("codelists/output/uk/query_*_snomed.csv") ) )
files


['codelists/output/uk/query_appendix_0_snomed.csv',
 'codelists/output/uk/query_appendix_1_snomed.csv',
 'codelists/output/uk/query_appendix_2_snomed.csv',
 'codelists/output/uk/query_appendix_3_snomed.csv',
 'codelists/output/uk/query_cataract_0_snomed.csv',
 'codelists/output/uk/query_cataract_1_snomed.csv',
 'codelists/output/uk/query_cataract_2_snomed.csv',
 'codelists/output/uk/query_cataract_3_snomed.csv',
 'codelists/output/uk/query_colorectal_0_snomed.csv',
 'codelists/output/uk/query_colorectal_1_snomed.csv',
 'codelists/output/uk/query_colorectal_2_snomed.csv',
 'codelists/output/uk/query_colorectal_3_snomed.csv',
 'codelists/output/uk/query_glaucoma_0_snomed.csv',
 'codelists/output/uk/query_glaucoma_1_snomed.csv',
 'codelists/output/uk/query_glaucoma_2_snomed.csv',
 'codelists/output/uk/query_glaucoma_3_snomed.csv',
 'codelists/output/uk/query_lung_0_snomed.csv',
 'codelists/output/uk/query_lung_1_snomed.csv',
 'codelists/output/uk/query_lung_2_snomed.csv',
 'codelists/outp

In [45]:
omop_concept_relationships[ (omop_concept_relationships['concept_id_1'].isin(omop_concepts_filtered['concept_id'])) & 
                            (omop_concept_relationships['relationship_id']=='Subsumes')  ]

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason


In [46]:
for f in files:
    name = re.split('/|\.', f)[-2]
    input_snomed_list = pd.read_csv(f)

    input_snomed_list['concept_id'] = input_snomed_list['concept_id'].astype(str)
    omop_concepts['concept_code'] = omop_concepts['concept_code'].astype(str)
    omop_concepts['concept_id'] = omop_concepts['concept_id'].astype(str)

    omop_concepts_filtered = omop_concepts[ omop_concepts['concept_code'].isin(input_snomed_list['concept_id']) ]

    omop_concepts_filtered = omop_concepts_filtered[['concept_id','concept_name','vocabulary_id','concept_code']]
    omop_concepts_filtered

    #omop_relationships_filtered = omop_concept_relationships[ omop_concept_relationships['relationship_id'].isin(['Maps to','Subsumes'])]

    omop_concept_relationships['concept_id_1'] = omop_concept_relationships['concept_id_1'].astype(str)
    omop_concept_relationships['concept_id_2'] = omop_concept_relationships['concept_id_2'].astype(str)

    concept_maps = omop_concepts_filtered.merge( omop_concept_relationships, how='left', left_on='concept_id', right_on='concept_id_1' )
    concept_maps_named = concept_maps.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id' )

    target_codes_icd10 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD10PCS' ]
    target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
    print( target_codes_icd10['relationship_id'].value_counts() )

    target_codes_icd9 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='ICD9Proc' ]
    target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
    print( target_codes_icd9['relationship_id'].value_counts() )

    target_codes_opcs4 = concept_maps_named[ concept_maps_named['vocabulary_id_y']=='OPCS4' ]
    target_codes_opcs4['opcs'] = target_codes_opcs4['concept_code_y'].str.replace('.','')

    print( target_codes_opcs4['relationship_id'].value_counts() )

    target_codes_icd9[['concept_code_y','concept_name_y','icd']].drop_duplicates().to_csv(OUTPUT_PATH+name+'_icd9.csv')
    target_codes_icd10[['concept_code_y','concept_name_y','icd']].drop_duplicates().to_csv(OUTPUT_PATH+name+'_icd10.csv')
    target_codes_opcs4[['concept_code_y','concept_name_y','opcs']].drop_duplicates().to_csv(OUTPUT_PATH+name+'_opcs.csv')

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:31: SettingWithCopyWarning: 
A value is trying 

relationship_id
Subsumes    75
Name: count, dtype: int64
relationship_id
Mapped from    21
Subsumes        8
Name: count, dtype: int64
relationship_id
Mapped from    50
Subsumes        8
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:31: SettingWithCopyWarning: 
A value is trying 

relationship_id
Subsumes    75
Name: count, dtype: int64
relationship_id
Mapped from    16
Subsumes        8
Name: count, dtype: int64
relationship_id
Mapped from    41
Subsumes        8
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:31: SettingWithCopyWarning: 
A value is trying 

...

/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:31: SettingWithCopyWarning: 
A value is trying 

relationship_id
Subsumes    4118
Name: count, dtype: int64
relationship_id
Mapped from    43
Subsumes       20
Name: count, dtype: int64
relationship_id
Mapped from    86
Subsumes       20
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:31: SettingWithCopyWarning: 
A value is trying 

relationship_id
Subsumes    3374
Name: count, dtype: int64
relationship_id
Mapped from    36
Subsumes       18
Name: count, dtype: int64
relationship_id
Mapped from    79
Subsumes       14
Name: count, dtype: int64
relationship_id
Subsumes    3341
Name: count, dtype: int64
relationship_id
Mapped from    33
Subsumes       18
Name: count, dtype: int64
relationship_id
Mapped from    64
Subsumes       14
Name: count, dtype: int64


/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd10['icd'] = target_codes_icd10['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_codes_icd9['icd'] = target_codes_icd9['concept_code_y']
/var/folders/jv/r9wkk1k157jgllfhz9lygd0r0000gp/T/ipykernel_31992/1691505545.py:31: SettingWithCopyWarning: 
A value is trying 

In [47]:
concept_maps_named

,concept_id_x,concept_name_x,vocabulary_id_x,concept_code_x,concept_id_1,concept_id_2,relationship_id,valid_start_date_x,valid_end_date_x,invalid_reason_x,concept_id_y,concept_name_y,domain_id,vocabulary_id_y,concept_class_id,standard_concept,concept_code_y,valid_start_date_y,valid_end_date_y,invalid_reason_y
0,4020571,Radionuclide therapy for glandular suppression,SNOMED,10492003,4020571,4220084,Using energy,20220128,20991231,NaN,4220084,Radiation,Observation,SNOMED,Physical Force,S,82107009,20020131,20991231,NaN
1,4020571,Radionuclide therapy for glandular suppression,SNOMED,10492003,4020571,40642538,Has status,20220128,20991231,NaN,40642538,Primitive,Metadata,SNOMED,Model Comp,NaN,900000000000074008,20020131,20991231,NaN
2,4020571,Radionuclide therapy for glandular suppression,SNOMED,10492003,4020571,40642539,Has Module,20220128,20991231,NaN,40642539,SNOMED CT core,Metadata,SNOMED,Model Comp,NaN,900000000000207008,20020131,20991231,NaN
3,4020571,Radionuclide therapy for glandular suppression,SNOMED,10492003,4020571,4020571,Mapped from,19700101,20991231,NaN,4020571,Radionuclide therapy for glandular suppression,Procedure,SNOMED,Procedure,S,10492003,20020131,20991231,NaN
4,4020571,Radionuclide therapy for glandular suppression,SNOMED,10492003,4020571,4020571,Maps to,19700101,20991231,NaN,4020571,Radionuclide therapy for glandular suppression,Procedure,SNOMED,Procedure,S,10492003,20020131,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10078,44812240,Ultrasound guided brachytherapy of cervix,SNOMED,911861000000109,44812240,4324450,Has intent,20150301,20991231,NaN,4324450,Guidance intent,Observation,SNOMED,Qualifier Value,S,429892002,20080731,20991231,NaN
10079,44812240,Ultrasound guided brachytherapy of cervix,SNOMED,911861000000109,44812240,4325271,Has dir proc site,20150301,20991231,NaN,4325271,Cervix uteri structure,Spec Anatomic Site,SNOMED,Body Structure,S,71252005,20020131,20991231,NaN
10080,44812240,Ultrasound guided brachytherapy of cervix,SNOMED,911861000000109,44812240,44812240,Mapped from,20140401,20991231,NaN,44812240,Ultrasound guided brachytherapy of cervix,Procedure,SNOMED,Procedure,S,911861000000109,20140401,20991231,NaN
10081,44812240,Ultrasound guided brachytherapy of cervix,SNOMED,911861000000109,44812240,44812240,Maps to,20140401,20991231,NaN,44812240,Ultrasound guided brachytherapy of cervix,Procedure,SNOMED,Procedure,S,911861000000109,20140401,20991231,NaN


In [48]:
target_codes_opcs4[['concept_code_y','concept_name_y','opcs']].drop_duplicates()

,concept_code_y,concept_name_y,opcs
749,E54.8,Other specified excision of lung,E548
750,E54.9,Unspecified excision of lung,E549
752,E54,Excision of lung,E54
963,Y89,Brachytherapy,Y89
2421,X65.4,Delivery of a fraction of external beam radiot...,X654
...,...,...,...
9925,E53.8,Other specified transplantation of lung,E538
9926,E53.9,Unspecified transplantation of lung,E539
9931,E53,Transplantation of lung,E53
9958,Y91.5,Megavoltage treatment for hypofractionated ste...,Y915


In [49]:
target_codes_opcs4

,concept_id_x,concept_name_x,vocabulary_id_x,concept_code_x,concept_id_1,concept_id_2,relationship_id,valid_start_date_x,valid_end_date_x,invalid_reason_x,...,concept_name_y,domain_id,vocabulary_id_y,concept_class_id,standard_concept,concept_code_y,valid_start_date_y,valid_end_date_y,invalid_reason_y,opcs
749,4000882,Lung excision,SNOMED,119746007,4000882,44509435,Subsumes,20180829,20991231,NaN,...,Other specified excision of lung,Procedure,OPCS4,Procedure,S,E54.8,19700101,20991231,NaN,E548
750,4000882,Lung excision,SNOMED,119746007,4000882,44509436,Mapped from,20180829,20991231,NaN,...,Unspecified excision of lung,Procedure,OPCS4,Procedure,NaN,E54.9,19700101,20991231,NaN,E549
752,4000882,Lung excision,SNOMED,119746007,4000882,46232360,Mapped from,20180829,20991231,NaN,...,Excision of lung,Procedure,OPCS4,Procedure,NaN,E54,20150401,20991231,NaN,E54
963,40317890,Brachytherapy,SNOMED,152198000,40317890,46233520,Mapped from,20190830,20991231,NaN,...,Brachytherapy,Procedure,OPCS4,Procedure,NaN,Y89,20150401,20991231,NaN,Y89
2421,4060641,X-ray beam therapy,SNOMED,169291001,4060641,44515661,Subsumes,20180829,20991231,NaN,...,Delivery of a fraction of external beam radiot...,Procedure,OPCS4,Procedure,S,X65.4,19700101,20991231,NaN,X654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9925,4337138,Transplant of lung,SNOMED,88039007,4337138,44509427,Subsumes,20180829,20991231,NaN,...,Other specified transplantation of lung,Procedure,OPCS4,Procedure,S,E53.8,19700101,20991231,NaN,E538
9926,4337138,Transplant of lung,SNOMED,88039007,4337138,44509428,Mapped from,20180829,20991231,NaN,...,Unspecified transplantation of lung,Procedure,OPCS4,Procedure,NaN,E53.9,19700101,20991231,NaN,E539
9931,4337138,Transplant of lung,SNOMED,88039007,4337138,46232359,Mapped from,20180829,20991231,NaN,...,Transplantation of lung,Procedure,OPCS4,Procedure,NaN,E53,20150401,20991231,NaN,E53
9958,44809885,Hypofractionated stereotactic radiotherapy usi...,SNOMED,895121000000103,44809885,44517141,Mapped from,20180829,20991231,NaN,...,Megavoltage treatment for hypofractionated ste...,Procedure,OPCS4,Procedure,NaN,Y91.5,19700101,20991231,NaN,Y915


In [50]:
omop_concept_relationships[omop_concept_relationships['concept_id_1']=='4096461']

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
624715,4096461,46274039,Value mapped from,20220801,20991231,NaN
11186518,4096461,40642537,Has status,20220128,20991231,NaN
11186519,4096461,40642539,Has Module,20220128,20991231,NaN
14623737,4096461,2002767,Mapped from,20180625,20991231,NaN
14623738,4096461,2002768,Subsumes,20180625,20991231,NaN
14623739,4096461,2002769,Subsumes,20180625,20991231,NaN
14623740,4096461,2002770,Subsumes,20180625,20991231,NaN
14623741,4096461,4018020,Concept was_a from,20140401,20991231,NaN
14623742,4096461,4018020,Mapped from,19700101,20991231,NaN
14623743,4096461,4018161,Concept was_a from,20140401,20991231,NaN


In [51]:
method_relationships = omop_concept_relationships[omop_concept_relationships['relationship_id']=='Has method']
method_relationships

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
14106,4237501,4045073,Has method,20220128,20991231,NaN
20970,619843,40293773,Has method,20220128,20991231,NaN
97320,36674184,4044176,Has method,20190131,20991231,NaN
97337,36674186,4081836,Has method,20190131,20991231,NaN
97339,36674186,4106470,Has method,20190131,20991231,NaN
...,...,...,...,...,...,...
18892362,619793,4044525,Has method,20220128,20991231,NaN
18892369,619794,4044525,Has method,20220128,20991231,NaN
18892385,619795,40293775,Has method,20220128,20991231,NaN
18892400,618616,40293773,Has method,20220128,20991231,NaN


In [52]:
snomed2opcs = target_codes_opcs4[['concept_id_x','concept_name_x','concept_name_y','concept_id_y','concept_code_y']]

snomed2opcs

m1 = snomed2opcs.merge(method_relationships, how='left', left_on='concept_id_x', right_on='concept_id_1')
m1
m2 = m1.merge(omop_concepts, how='left', left_on='concept_id_2', right_on='concept_id')
m2

methods = m2[['concept_name_x','concept_name_y','concept_code_y','concept_name']]

In [53]:
opcs2cat = pd.DataFrame(data={
    'opcs_name': methods['concept_name_y'],
    'opcs_code': methods['concept_code_y'],
    'category': methods['concept_name']
}).drop_duplicates()
opcs2cat

,opcs_name,opcs_code,category
0,Other specified excision of lung,E54.8,Excision - action
1,Unspecified excision of lung,E54.9,Excision - action
2,Excision of lung,E54,Excision - action
3,Brachytherapy,Y89,Brachytherapy - action
4,Delivery of a fraction of external beam radiot...,X65.4,NaN
...,...,...,...
75,Other specified transplantation of lung,E53.8,Surgical transplantation - action
76,Unspecified transplantation of lung,E53.9,Surgical transplantation - action
77,Transplantation of lung,E53,Surgical transplantation - action
78,Megavoltage treatment for hypofractionated ste...,Y91.5,NaN


In [54]:
opcs2cat['opcs_code'].value_counts()

opcs_code
M70.6    2
X37.3    2
E54.8    1
Y92.1    1
X65.7    1
        ..
E53.2    1
X65      1
X65.9    1
X65.8    1
E57.1    1
Name: count, Length: 78, dtype: int64